The Team (5 points)

- Auston Ng, 301565494
- Minh Hoang Le (Ted), 301477912

Schema

In [1]:
%load_ext sql
%config SqlMagic.displaylimit = None

displaylimit: Value None will be treated as 0 (no limit)

In [2]:
%sql sqlite:///library.db

Connecting to 'sqlite:///library.db'

In [5]:
%%sql
-- Drop all tables in reverse dependency order

-- First, drop tables that depend on other tables (have foreign keys)
DROP TABLE IF EXISTS eventRegistration;
DROP TABLE IF EXISTS event;
DROP TABLE IF EXISTS fine;
DROP TABLE IF EXISTS loan;
DROP TABLE IF EXISTS personnel;
DROP TABLE IF EXISTS item_category;
DROP TABLE IF EXISTS item_author;

-- Then drop the independent tables (referenced by foreign keys)
DROP TABLE IF EXISTS eventType;
DROP TABLE IF EXISTS room;
DROP TABLE IF EXISTS member;
DROP TABLE IF EXISTS category;
DROP TABLE IF EXISTS author;
DROP TABLE IF EXISTS libraryItem;

Running query in 'sqlite:///library.db'

++
||
++
++

- NOTE: date and time will be expressed in ISO 8601 format for consistency (YYYY/MM/DD, i.e. 2022/01/05)

In [8]:
%%sql
-- highest level general abstraction that all library items fit into 
CREATE TABLE libraryItem (
    id INTEGER,
    title TEXT NOT NULL,
    publicationDate DATE NOT NULL CHECK(date(publicationDate) is not NULL),
    itemType TEXT NOT NULL,  -- e.g. 'book', 'magazine', 'cd', etc.
    status TEXT NOT NULL DEFAULT 'available' CHECK(status IN ('available', 'unavailable')),
    acquisitionStatus TEXT NOT NULL DEFAULT 'owned' CHECK(acquisitionStatus IN ('owned', 'planned')),
    PRIMARY KEY(id)
);

-- intermediary tables for all many to many relationships
CREATE TABLE author(
    id INTEGER,
    firstName TEXT NOT NULL,
    lastName TEXT NOT NULL,
    PRIMARY KEY(id)
);

CREATE TABLE category(
    id INTEGER,
    categoryName TEXT NOT NULL, -- fiction, music, action, romance, teen, children, all ages, adult, etc.
    PRIMARY KEY(id)
);

CREATE TABLE item_author(
    itemid INTEGER NOT NULL,
    authorid INTEGER NOT NULL,
    PRIMARY KEY(itemid, authorid),
    FOREIGN KEY(itemid) REFERENCES libraryItem(id),
    FOREIGN KEY(authorid) REFERENCES author(id)
);


CREATE TABLE item_category(
    itemid INTEGER NOT NULL,
    categoryid INTEGER NOT NULL,
    PRIMARY KEY(itemid, categoryid),
    FOREIGN KEY(itemid) REFERENCES libraryItem(id),
    FOREIGN KEY(categoryid) REFERENCES category(id)
);
--
CREATE TABLE member (
    id INTEGER,
    firstName TEXT NOT NULL,
    lastName TEXT NOT NULL,
    email TEXT NOT NULL UNIQUE,
    joinDate DATE DEFAULT CURRENT_DATE CHECK(joinDate = date(joinDate)),
    PRIMARY KEY(id)
);

CREATE TABLE loan(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    itemid INTEGER NOT NULL,
    memberid INTEGER NOT NULL,
    loanDate DATE NOT NULL DEFAULT CURRENT_DATE CHECK(loanDate = date(loanDate)),
    dueDate DATE NOT NULL DEFAULT (date('now', '+14 days')),
    returnedDate DATE,
    FOREIGN KEY(itemid) REFERENCES libraryItem(id),
    FOREIGN KEY(memberid) REFERENCES member(id)
    CHECK(dueDate >= loanDate),
    CHECK(returnedDate IS NULL OR returnedDate >= loanDate)
);


CREATE TABLE fine(
    id INTEGER,
    loanid INTEGER NOT NULL,
    fineDate DATE,
    fineAmount DECIMAL(18,2),
    isPaid BOOLEAN DEFAULT FALSE,
    paymentDate DATE,
    PRIMARY KEY(id),
    FOREIGN KEY(loanid) REFERENCES loan(id)
    CHECK(paymentDate IS NULL OR (isPaid = TRUE AND paymentDate >= fineDate))
);


CREATE TABLE personnel (
    id INTEGER,
    memberid INTEGER UNIQUE NOT NULL, 
    hireDate DATE DEFAULT CURRENT_DATE,
    salary INTEGER DEFAULT 0, 
    jobTitle TEXT NOT NULL CHECK(jobTitle IN ('librarian', 'volunteer', 'admin')), 
    phone TEXT,
    PRIMARY KEY(id),
    FOREIGN KEY(memberid) REFERENCES member(id)
);

CREATE TABLE room (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,       -- e.g. "Main Hall", "Room A", "Community Room"
    capacity INTEGER NOT NULL
);

CREATE TABLE eventType (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL UNIQUE  -- e.g. 'Book Club', 'Art Show', etc.
);
    
CREATE TABLE event (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    eventDate DATE NOT NULL CHECK(eventDate = date(eventDate)),
    description TEXT,
    targetAudience TEXT,  -- or FK to category
    eventTypeid INTEGER NOT NULL,
    roomid INTEGER NOT NULL,
    hostedByPersonnelID INTEGER, -- the personnel responsible for hosting the event 
    FOREIGN KEY(eventTypeid) REFERENCES eventType(id),
    FOREIGN KEY(roomid) REFERENCES room(id),
    FOREIGN KEY(hostedByPersonnelid) REFERENCES personnel(id)
    CHECK(eventDate >= CURRENT_DATE)
);

CREATE TABLE eventRegistration (
    eventid INTEGER NOT NULL,
    memberid INTEGER NOT NULL,
    registrationDate DATE NOT NULL DEFAULT CURRENT_DATE,
    PRIMARY KEY(eventid, memberid),
    FOREIGN KEY(eventid) REFERENCES event(id),
    FOREIGN KEY(memberid) REFERENCES member(id)
);




Running query in 'sqlite:///library.db'

++
||
++
++

In [10]:
%%sql 
-- Triggers for updating item status: Available, Unavailable 
CREATE TRIGGER set_item_unavailable
AFTER INSERT ON loan
FOR EACH ROW
WHEN NEW.returnedDate is NULL 
BEGIN
    UPDATE libraryItem SET status = 'unavailable' where id=NEW.itemid;
END;

CREATE TRIGGER set_item_available
AFTER UPDATE ON loan 
FOR EACH ROW
WHEN NEW.returnedDate is not NULL and old.returnedDate is NULL
BEGIN
    UPDATE libraryItem SET status = 'available' where id=new.itemid;

END;


-- Triggers for generating fine when returning an item that is past the due date
CREATE TRIGGER generate_fine_on_late_return
AFTER UPDATE ON loan
FOR EACH ROW
WHEN NEW.returnedDate IS NOT NULL AND NEW.returnedDate > NEW.dueDate
BEGIN
    INSERT INTO fine (loanid, fineDate, fineAmount, isPaid)
    VALUES (NEW.id, NEW.returnedDate, 
            (julianday(NEW.returnedDate) - julianday(NEW.dueDate)) * 0.50, -- $0.50 per day
            FALSE);
END;

-- Trigger that checks for a maximum loan constraint a person can have, which is 2 item
CREATE TRIGGER max_active_loans_check
BEFORE INSERT ON loan
FOR EACH ROW
BEGIN
    SELECT CASE
        WHEN (SELECT COUNT(*) FROM loan 
              WHERE memberid = NEW.memberid AND returnedDate IS NULL) >= 2
        THEN RAISE(ABORT, 'Member has reached maximum number of active loans (2)')
    END;
END;


CREATE TRIGGER prevent_loan_with_unpaid_fines
BEFORE INSERT ON loan
FOR EACH ROW
BEGIN
    SELECT CASE
        WHEN EXISTS (
            SELECT 1 FROM fine f
            JOIN loan l ON f.loanid = l.id
            WHERE l.memberid = NEW.memberid AND f.isPaid = FALSE
        )
        THEN RAISE(ABORT, 'Cannot issue loan to member with unpaid fines')
    END;
END;


CREATE TRIGGER prevent_loan_unavailable_item
BEFORE INSERT ON loan
FOR EACH ROW
BEGIN
    SELECT CASE
        WHEN EXISTS (
            SELECT 1 FROM libraryItem WHERE id = NEW.itemid AND status = 'unavailable'
        )
        THEN RAISE(ABORT, 'This item is currently unavailable')
    END;
END;

Running query in 'sqlite:///library.db'

++
||
++
++

In [12]:
%%sql
-- Populate author table with diverse creators
INSERT INTO author (id, firstName, lastName) VALUES
-- Book Authors
(1, 'Toni', 'Morrison'),
(2, 'Gabriel', 'García Márquez'),
(3, 'Neil', 'Gaiman'),
(4, 'Octavia', 'Butler'),
(5, 'Haruki', 'Murakami'),
-- Musicians/Composers
(6, 'Ludwig van', 'Beethoven'),
(7, 'Beyoncé', 'Knowles-Carter'),
(8, 'Kendrick', 'Lamar'),
(9, 'Miles', 'Davis'),
-- Film Directors
(10, 'Bong', 'Joon-ho'),
(11, 'Ava', 'DuVernay'),
(12, 'Alfonso', 'Cuarón'),
-- Academic Authors
(13, 'Yuval Noah', 'Harari'),
(14, 'Michelle', 'Alexander'),
(15, 'Carl', 'Sagan');

Running query in 'sqlite:///library.db'

15 rows affected.

++
||
++
++

In [15]:
%%sql
SELECT * FROM author;

Running query in 'sqlite:///library.db'

id,firstName,lastName
1,Toni,Morrison
2,Gabriel,García Márquez
3,Neil,Gaiman
4,Octavia,Butler
5,Haruki,Murakami
6,Ludwig van,Beethoven
7,Beyoncé,Knowles-Carter
8,Kendrick,Lamar
9,Miles,Davis
10,Bong,Joon-ho


In [17]:
%%sql
-- Populate category table with diverse categories
INSERT INTO category (id, categoryName) VALUES
-- Book/Text Categories
(1, 'Fiction'),
(2, 'Non-fiction'),
(3, 'Fantasy'),
(4, 'Science Fiction'),
(5, 'Mystery'),
(6, 'Biography'),
(7, 'History'),
(8, 'Romance'),
(9, 'Horror'),
-- Music Categories
(10, 'Classical'),
(11, 'Rock'),
(12, 'Hip-Hop'),
(13, 'Jazz'),
(14, 'Electronic'),
-- Age Groups
(15, 'Children'),
(16, 'Young Adult'),
(17, 'Adult'),
-- Format Categories
(18, 'Graphic Novel'),
(19, 'Periodical'),
(20, 'Interactive'),
(21, 'Language Learning');

Running query in 'sqlite:///library.db'

21 rows affected.

++
||
++
++

In [19]:
%%sql 
SELECT *  FROM category;

Running query in 'sqlite:///library.db'

id,categoryName
1,Fiction
2,Non-fiction
3,Fantasy
4,Science Fiction
5,Mystery
6,Biography
7,History
8,Romance
9,Horror
10,Classical


In [21]:
%%sql
-- Populate libraryItem table with diverse items
INSERT INTO libraryItem (id, title, publicationDate, itemType, status, acquisitionStatus) VALUES
-- Books (Print)
(1, 'Beloved', '1987-09-02', 'print book', 'available', 'owned'),
(2, 'One Hundred Years of Solitude', '1967-05-30', 'print book', 'available', 'owned'),
(3, 'American Gods', '2001-06-19', 'print book', 'available', 'owned'),
(4, 'Parable of the Sower', '1993-10-01', 'print book', 'available', 'owned'),
(5, 'Norwegian Wood', '1987-08-04', 'print book', 'available', 'owned'),
-- E-books
(6, 'Sapiens: A Brief History of Humankind', '2011-01-01', 'online book', 'available', 'owned'),
(7, 'The New Jim Crow', '2010-01-05', 'online book', 'available', 'owned'),
(8, 'Cosmos', '1980-09-10', 'online book', 'available', 'owned'),
-- Music CDs
(9, 'Symphony No. 9', '1995-03-15', 'cd', 'available', 'owned'),
(10, 'Lemonade', '2016-04-23', 'cd', 'available', 'owned'),
(11, 'To Pimp a Butterfly', '2015-03-15', 'cd', 'available', 'owned'),
(12, 'Kind of Blue', '1959-08-17', 'cd', 'available', 'owned'),
-- Vinyl Records
(13, 'Abbey Road', '1969-09-26', 'record', 'available', 'owned'),
(14, 'Parasite', '2019-05-30', 'record', 'available', 'owned'),
(15, '13th', '2016-10-07', 'record', 'available', 'owned'),
(16, 'Roma', '2018-11-21', 'record', 'available', 'owned'),
-- Magazines
(17, 'National Geographic: Ocean Life', '2023-01-15', 'magazine', 'unavailable', 'owned'),
(18, 'Scientific American: AI Revolution', '2023-03-01', 'magazine', 'unavailable', 'owned'),
--Planned
(19, 'The Ministry of the Future', '2020-10-06', 'online book', 'unavailable', 'planned'),
(20, 'Renaissance', '2022-07-29', 'cd', 'available', 'planned');


Running query in 'sqlite:///library.db'

20 rows affected.

++
||
++
++

In [23]:
%%sql 
SELECT * FROM libraryItem;

Running query in 'sqlite:///library.db'

id,title,publicationDate,itemType,status,acquisitionStatus
1,Beloved,1987-09-02,print book,available,owned
2,One Hundred Years of Solitude,1967-05-30,print book,available,owned
3,American Gods,2001-06-19,print book,available,owned
4,Parable of the Sower,1993-10-01,print book,available,owned
5,Norwegian Wood,1987-08-04,print book,available,owned
6,Sapiens: A Brief History of Humankind,2011-01-01,online book,available,owned
7,The New Jim Crow,2010-01-05,online book,available,owned
8,Cosmos,1980-09-10,online book,available,owned
9,Symphony No. 9,1995-03-15,cd,available,owned
10,Lemonade,2016-04-23,cd,available,owned


In [25]:
%%sql
-- Populate item_category table with relationships
INSERT INTO item_category (itemid, categoryid) VALUES
-- Print books with categories
(1, 1),  -- Beloved: Fiction
(1, 17), -- Beloved: Adult
(2, 1),  -- One Hundred Years of Solitude: Fiction
(2, 3),  -- One Hundred Years of Solitude: Fantasy
(3, 1),  -- American Gods: Fiction
(3, 3),  -- American Gods: Fantasy
(3, 17), -- American Gods: Adult
(4, 1),  -- Parable of the Sower: Fiction
(4, 4),  -- Parable of the Sower: Science Fiction
(5, 1),  -- Norwegian Wood: Fiction
(5, 8),  -- Norwegian Wood: Romance

-- Online books with categories
(6, 2),  -- Sapiens: Non-fiction
(6, 7),  -- Sapiens: History
(7, 2),  -- The New Jim Crow: Non-fiction
(8, 2),  -- Cosmos: Non-fiction

-- Music with genres
(9, 10),  -- Symphony No. 9: Classical
(10, 12), -- Lemonade: Hip-Hop
(10, 14), -- Lemonade: Electronic
(11, 12), -- To Pimp a Butterfly: Hip-Hop
(12, 13), -- Kind of Blue: Jazz
(13, 11), -- Abbey Road: Rock

-- Records (which appear to be films based on titles)
(14, 5),  -- Parasite: Mystery
(15, 2),  -- 13th: Non-fiction
(16, 1),  -- Roma: Fiction

-- Magazines
(17, 2),  -- National Geographic: Non-fiction
(17, 19), -- National Geographic: Periodical
(18, 2),  -- Scientific American: Non-fiction
(18, 19), -- Scientific American: Periodical

-- Planned acquisitions
(19, 1),  -- The Ministry of the Future: Fiction
(19, 4),  -- The Ministry of the Future: Science Fiction
(20, 12), -- Renaissance: Hip-Hop
(20, 14); -- Renaissance: Electronic

Running query in 'sqlite:///library.db'

32 rows affected.

++
||
++
++

In [27]:
%%sql 
SELECT * from item_category;

Running query in 'sqlite:///library.db'

itemid,categoryid
1,1
1,17
2,1
2,3
3,1
3,3
3,17
4,1
4,4
5,1


In [29]:
%%sql
-- Populate item_author table
INSERT INTO item_author (itemid, authorid) VALUES
-- Books
(1, 1),  -- Beloved by Toni Morrison
(2, 2),  -- One Hundred Years of Solitude by Gabriel García Márquez
(3, 3),  -- American Gods by Neil Gaiman
(4, 4),  -- Parable of the Sower by Octavia Butler
(5, 5),  -- Norwegian Wood by Haruki Murakami

-- Online books
(6, 13), -- Sapiens by Yuval Noah Harari
(7, 14), -- The New Jim Crow by Michelle Alexander
(8, 15), -- Cosmos by Carl Sagan

-- Music CDs
(9, 6),  -- Symphony No. 9 by Beethoven
(10, 7), -- Lemonade by Beyoncé
(11, 8), -- To Pimp a Butterfly by Kendrick Lamar
(12, 9), -- Kind of Blue by Miles Davis

-- Records (which appear to be films/media based on titles)
(14, 10), -- Parasite by Bong Joon-ho
(15, 11), -- 13th by Ava DuVernay
(16, 12), -- Roma by Alfonso Cuarón

-- Planned acquisitions
(19, 4),  -- The Ministry of the Future by Octavia Butler (assuming)
(20, 7);  -- Renaissance by Beyoncé

Running query in 'sqlite:///library.db'

17 rows affected.

++
||
++
++

In [31]:
%%sql 
SELECT * FROM item_author;

Running query in 'sqlite:///library.db'

itemid,authorid
1,1
2,2
3,3
4,4
5,5
6,13
7,14
8,15
9,6
10,7


In [33]:
%%sql
-- Populate member table
INSERT INTO member (id, firstName, lastName, email, joinDate) VALUES
(1, 'Jacking', 'Jack', 'jj@email.com', '2025-03-15'),
(2, 'Mikhail', 'Williams', 'mwilliams@email.com', '2024-05-07'),
(3, 'Sarah', 'Davis', 'idontlikecats@email.com', '2023-09-22'),
(4, 'Ted', 'Le', 'ted@email.com', '2025-01-30'),
(5, 'Austin', 'Ng', 'austin@email.com', '2024-06-12');

Running query in 'sqlite:///library.db'

5 rows affected.

++
||
++
++

In [35]:
%%sql
SELECT * FROM member;


Running query in 'sqlite:///library.db'

id,firstName,lastName,email,joinDate
1,Jacking,Jack,jj@email.com,2025-03-15
2,Mikhail,Williams,mwilliams@email.com,2024-05-07
3,Sarah,Davis,idontlikecats@email.com,2023-09-22
4,Ted,Le,ted@email.com,2025-01-30
5,Austin,Ng,austin@email.com,2024-06-12


In [38]:
%%sql
-- Populate personnel table
INSERT INTO personnel (id, memberid, hireDate, salary, jobTitle, phone) VALUES
(1, 1, '2021-11-15', 46000, 'librarian', '604-555-2345'),
(2, 5, '2024-06-10', 0, 'volunteer', '604-555-1234'),
(3, 4, '2024-11-15', 0, 'volunteer', '727-555-2345');

Running query in 'sqlite:///library.db'

3 rows affected.

++
||
++
++

In [40]:
%%sql 
SELECT * FROM personnel;

Running query in 'sqlite:///library.db'

id,memberid,hireDate,salary,jobTitle,phone
1,1,2021-11-15,46000,librarian,604-555-2345
2,5,2024-06-10,0,volunteer,604-555-1234
3,4,2024-11-15,0,volunteer,727-555-2345


In [42]:
%%sql
-- Populate loan table
INSERT INTO loan (id, itemid, memberid, loanDate, dueDate, returnedDate) VALUES
(1, 1, 2, '2025-03-01', '2025-03-15', '2025-03-14');

Running query in 'sqlite:///library.db'

1 rows affected.

++
||
++
++

In [44]:
%%sql
SELECT * FROM loan;

Running query in 'sqlite:///library.db'

id,itemid,memberid,loanDate,dueDate,returnedDate
1,1,2,2025-03-01,2025-03-15,2025-03-14


In [46]:
%%sql
SELECT * FROM fine;

Running query in 'sqlite:///library.db'

id,loanid,fineDate,fineAmount,isPaid,paymentDate


In [48]:
%%sql
-- Populate room table
INSERT INTO room (id, name, capacity) VALUES
(1, 'Main Reading Room', 40),
(2, 'Childrens Area', 20),
(3, 'Conference Room', 25),
(4, 'Multimedia Lab', 15),
(5, 'Study Room', 8);

Running query in 'sqlite:///library.db'

5 rows affected.

++
||
++
++

In [50]:
%%sql 
SELECT * FROM room;

Running query in 'sqlite:///library.db'

id,name,capacity
1,Main Reading Room,40
2,Childrens Area,20
3,Conference Room,25
4,Multimedia Lab,15
5,Study Room,8


In [52]:
%%sql
-- Populate eventType table
INSERT INTO eventType (id, name) VALUES
(1, 'Book Club'),
(2, 'Author Talk'),
(3, 'Workshop'),
(4, 'Film Screening'),
(5, 'Story Time');

Running query in 'sqlite:///library.db'

5 rows affected.

++
||
++
++

In [54]:
%%sql
SELECT * FROM eventType;


Running query in 'sqlite:///library.db'

id,name
1,Book Club
2,Author Talk
3,Workshop
4,Film Screening
5,Story Time


In [56]:
%%sql
-- Populate event table
INSERT INTO event (id, name, eventDate, description, targetAudience, eventTypeid, roomid, hostedByPersonnelID) VALUES
(1, 'Fixion Discussion Day', '2025-04-21', 'Monthly discussion a chosen fiction book. This month: Animal Farm', 'Adults', 1, 1, 4),
(2, 'Mum! what on Earth?', '2025-04-6', 'Story time for mum and child (Below 10)', 'Parents and children', 5, 2, 5);

Running query in 'sqlite:///library.db'

2 rows affected.

++
||
++
++

In [58]:
%%sql 
SELECT * FROM event;

Running query in 'sqlite:///library.db'

id,name,eventDate,description,targetAudience,eventTypeid,roomid,hostedByPersonnelID
1,Fixion Discussion Day,2025-04-21,Monthly discussion a chosen fiction book. This month: Animal Farm,Adults,1,1,4
2,Mum! what on Earth?,2025-04-6,Story time for mum and child (Below 10),Parents and children,5,2,5


In [60]:
%%sql
-- Populate eventRegistration table
INSERT INTO eventRegistration (eventid, memberid, registrationDate) VALUES
(1, 2, '2025-03-31'),
(1, 3, '2025-03-29');

Running query in 'sqlite:///library.db'

2 rows affected.

++
||
++
++

In [62]:
%%sql 
SELECT * FROM eventRegistration;

Running query in 'sqlite:///library.db'

eventid,memberid,registrationDate
1,2,2025-03-31
1,3,2025-03-29
